# Few Shot Learning

## Imports

In [12]:
from bert import BERT
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

from data_cleaning import Data_Preprocessing
from arabert.preprocess import ArabertPreprocessor

# Core
import random

# Basics
import numpy as np
import pandas as pd
import torch

# Metrics
from sklearn.metrics import *

# Tokeniser
from transformers import BertTokenizer

# Utility
from tqdm import tqdm

# Dataloader
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Scheduler
from transformers import get_linear_schedule_with_warmup

# Optimiser
from transformers import AdamW

# Model
from transformers import BertForSequenceClassification
import torch.nn as nn

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Model

In [11]:
class HateBERT_FewShot:
    def __init__(self,args):
        # fix the random
        random.seed(args['seed_val'])
        np.random.seed(args['seed_val'])
        torch.manual_seed(args['seed_val'])
        torch.cuda.manual_seed_all(args['seed_val'])
        
        # set device
        self.device = torch.device(args['device'])

        self.weights=args['weights']
        
        # initiliase tokeniser
        self.tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english")

        self.model_save_path = args['model_save_path']
        self.name = args['name']
        
    ##-----------------------------------------------------------##
    ##----------------- Utility Functions -----------------------##
    ##-----------------------------------------------------------##
    def encode(self,data,max_len):
        input_ids = []
        attention_masks = []
        for sent in tqdm(data):
            # use in-built tokeniser of Bert
            encoded_dict = self.tokenizer.encode_plus(
                            sent,
                            add_special_tokens =True, # for [CLS] and [SEP]
                            max_length = max_len,
                            truncation = True,
                            padding = 'max_length',
                            return_attention_mask = True,
                            return_tensors = 'pt', # return pytorch tensors
            )
            input_ids.append(encoded_dict['input_ids'])
            # attention masks notify where padding has been added 
            # and where is the sentence
            attention_masks.append(encoded_dict['attention_mask'])
        
        return [input_ids,attention_masks]
    
    ##-----------------------------------------------------------##
    ##------------------ Dataloader -----------------------------##
    ##-----------------------------------------------------------##
    def get_dataloader(self,samples, batch_size,is_train=False):
        inputs,masks,labels = samples

        # Convert the lists into tensors.
        inputs = torch.cat(inputs, dim=0)
        masks = torch.cat(masks, dim=0)
        labels = torch.tensor(labels)

        # convert to dataset
        data = TensorDataset(inputs,masks,labels)

        if(is_train==False):
            # use random sampler for training to shuffle
            # train data
            sampler = SequentialSampler(data)
        else:
            # order does not matter for validation as we just 
            # need the metrics
            sampler = RandomSampler(data)  

        dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size,drop_last=True)

        return dataloader
    
    ##-----------------------------------------------------------##
    ##----------------- Training Utilities ----------------------##
    ##-----------------------------------------------------------## 
    def get_optimiser(self,learning_rate,model):
        # using AdamW optimiser from transformers library
        return AdamW(model.parameters(),
                  lr = learning_rate, 
                  eps = 1e-8
                )
    
    def get_scheduler(self,epochs,optimiser,train_dl):
        total_steps = len(train_dl) * epochs
        return get_linear_schedule_with_warmup(optimiser, 
                num_warmup_steps = 0, 
                num_training_steps = total_steps)
    
    def evalMetric(self, y_true, y_pred, prefix):
        # calculate all the metrics and add prefix to them
        # before saving in dictionary
        accuracy = accuracy_score(y_true, y_pred)
        mf1Score = f1_score(y_true, y_pred, average='macro')
        f1Score = f1_score(y_true, y_pred)
        area_under_c = roc_auc_score(y_true, y_pred)
        recallScore = recall_score(y_true, y_pred)
        precisionScore = precision_score(y_true, y_pred)

        nonhate_f1Score = f1_score(y_true, y_pred, pos_label=0)
        non_recallScore = recall_score(y_true, y_pred, pos_label=0)
        non_precisionScore = precision_score(y_true, y_pred, pos_label=0)
        return {prefix+"accuracy": accuracy, prefix+'mF1Score': mf1Score, 
            prefix+'f1Score': f1Score, prefix+'auc': area_under_c,
            prefix+'precision': precisionScore, 
            prefix+'recall': recallScore, 
            prefix+'non_hatef1Score': nonhate_f1Score, 
            prefix+'non_recallScore': non_recallScore, 
            prefix+'non_precisionScore': non_precisionScore}
    
    ##-----------------------------------------------------------##
    ##---------------- Different Train Loops --------------------##
    ##-----------------------------------------------------------## 
    def evaluate(self,model,loader,which):
        # to evaluate model on test and validation set

        model.eval() # put model in eval mode

        # maintain total loss to save in metrics
        total_eval_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(loader):
            # separate input, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            with torch.no_grad(): # do not construct compute graph
                outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
            # output is always a tuple, thus we have to 
            # separate it manually
            loss = outputs[0]
            logits = outputs[1]

            # add the current loss
            # loss.item() extracts loss value as a float
            total_eval_loss += loss.item()

            # calculate true labels and convert it into numpy array
            b_y_true = b_labels.cpu().data.squeeze().numpy()
            
            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

        # calculate metrics
        metrics = self.evalMetric(y_true,y_pred,which+"_")

        # Calculate the average loss over all of the batches.
        avg_loss = total_eval_loss / len(loader)
        # add it to the metric
        metrics[which+'_avg_loss'] = avg_loss

        return metrics
    
    
    def run_train_loop(self,model,train_loader,optimiser,scheduler):

        model.train() # put model in train mode

        # maintain total loss to add to metric
        total_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(train_loader):
            # separate inputs, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            # Ref: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch#:~:text=In%20PyTorch%20%2C%20we%20need%20to,backward()%20call.
            model.zero_grad()                

            outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

            # outputs is always returned as tuple
            # Separate it manually
            logits = outputs[1]

            # define new loss function so that we can include
            # weights
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(
                        self.weights,dtype=torch.float)).to(self.device)
            
            loss = loss_fct(logits,b_labels)
            
            # calculate current loss
            # loss.item() extracts loss value as a float
            total_loss += loss.item()

            # Back-propagation
            loss.backward()

            # calculate true labels
            b_y_true = b_labels.cpu().data.squeeze().numpy()

            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

            # clip gradient to prevent exploding gradient
            # problems
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # gradient descent
            optimiser.step()
            
            # schedule learning rate accordingly
            scheduler.step()

        # calculate avg loss 
        avg_train_loss = total_loss / len(train_loader)

        # calculate metrics
        train_metrics = self.evalMetric(y_true,y_pred,"Train_")
        
        # print results
        print('avg_train_loss',avg_train_loss)
        print('train_f1Score',train_metrics['Train_f1Score'])
        print('train_accuracy',train_metrics['Train_accuracy'])

        # add loss to metrics
        train_metrics['Train_avg_loss'] = avg_train_loss

        return train_metrics
    
    
    ##------------------------------------------------------------##
    ##----------------- Main Train Loop --------------------------##
    ##------------------------------------------------------------##
    def train(self,model,data_loaders,optimiser,scheduler,epochs,save_model):
        # save train stats per epoch
        train_stats = []
        test_stats=[]
        best_test={}
        train_loader,val_loader,test_loader = data_loaders
        # maintain best mF1 Score to save best model
        best_mf1Score=-1.0
        for epoch_i in range(0, epochs):
            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            
            print("")
            print('Training...')
            # run trian loop
            train_metrics = self.run_train_loop(model,train_loader,
                                            optimiser,scheduler)

            print("")
            print("Running Validation...") 
            # test on validation set
            val_metrics = self.evaluate(model,val_loader,"Val")
            
            print("Validation Loss: ",val_metrics['Val_avg_loss'])
            print("Validation Accuracy: ",val_metrics['Val_accuracy'])
            
            stats = {}

            # evaluate model on alternate epochs
#             if(val_metrics['Val_mF1Score']>best_mf1Score):
            if(epoch_i%2==1):
                print("Testing Model....")
#                 best_mf1Score=val_metrics['Val_mF1Score']
                if(save_model):
                    torch.save(model.state_dict(), self.model_save_path+
                        '/best_bert_'+self.name+'.pt')
                # evaluate model on test set
                test_metrics = self.evaluate(model,test_loader,'')
                test_metrics['name']=self.name+'-'+str(epoch_i)
                test_stats.append(test_metrics)
            
            if(val_metrics['Val_mF1Score']>best_mf1Score):
                print("Best mF1Score....")
                best_mf1Score=val_metrics['Val_mF1Score']
                if(save_model):
                    torch.save(model.state_dict(), self.model_save_path+
                        '/best_bert_'+self.name+'.pt')
                # evaluate model on test set
                best_test = self.evaluate(model,test_loader,'')
                best_test['name']=self.name+'-'+str(epoch_i)+'_best'


            stats['epoch']=epoch_i+1

            # add train and val metrics of the epoch to 
            # same dictionary
            stats.update(train_metrics)
            stats.update(val_metrics)

            train_stats.append(stats)
            
        test_stats.append(best_test)
        return train_stats,test_stats
    
    ##-----------------------------------------------------------##
    ##----------------------- Main Pipeline ---------------------##
    ##-----------------------------------------------------------##
    def run(self,args,df_train,df_val,df_test):
        # get X and Y data points 
        X_train = df_train['Text'].values
        Y_train = df_train['Label'].values
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values
        X_val = df_val['Text'].values
        Y_val = df_val['Label'].values
        
        # encode data
        # returns list of data and attention masks
        train_data = self.encode(X_train,args['max_len'])
        val_data = self.encode(X_val,args['max_len'])
        test_data = self.encode(X_test,args['max_len'])
        
        # add labels to data so that we can send them to
        # dataloader function together
        train_data.append(Y_train)
        val_data.append(Y_val)
        test_data.append(Y_test)
        
        # convert to dataloader
        train_dl =self.get_dataloader(train_data,args['batch_size'],True)
        val_dl =self.get_dataloader(val_data,args['batch_size'])                          
        test_dl =self.get_dataloader(test_data,args['batch_size'])
        
        # intialise model
#         model = BertForSequenceClassification.from_pretrained(
#                 args['bert_model'], 
#                 num_labels = 2, 
#                 output_attentions = False, # Whether the model returns attentions weights.
#                 output_hidden_states = False, # Whether the model returns all hidden-states.
#             )
#         model = self.load_model(args['model_path'],args)
        model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english")
        model.to(self.device)
        
        optimiser = self.get_optimiser(args['learning_rate'],model)
        
        scheduler = self.get_scheduler(args['epochs'],optimiser,train_dl)
        
        # Run train loop and evaluate on validation data set
        # on each epoch. Store best model from all epochs 
        # (best mF1 Score on Val set) and evaluate it on
        # test set
        train_stats,test_stats = self.train(model,[train_dl,val_dl,test_dl],
                                optimiser,scheduler,args['epochs'],args['save_model'])
        
        return train_stats,test_stats
        
    ##-----------------------------------------------------------##
    ##-------------------- Other Utilities ----------------------##
    ##-----------------------------------------------------------##
    def run_test(self,model,df_test,args):
        # to evaluate test set on the final saved model
        # to retrieve results if necessary
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values

        test_data = self.encode(X_test,args['max_len'])

        test_data.append(Y_test)

        test_dl =self.get_dataloader(test_data,32)

        metrics = self.evaluate(model,test_dl,"Test")

        return metrics
    
    def load_model(self,path,args):
        # load saved best model
        saved_model = BertForSequenceClassification.from_pretrained(
                args['bert_model'], 
                num_labels = 2, 
                output_attentions = False, # Whether the model returns attentions weights.
                output_hidden_states = False, # Whether the model returns all hidden-states.
            )
        
        saved_model.load_state_dict(torch.load(path))
        
        return saved_model

## Utility Functions

In [1]:
def load_dataset_part(args,index,seed,train_cnt):
    # initialise constants 
    path = args['data_path']
    # read dataframes
    df_train = pd.read_csv(path+'train_'+str(index)+'.csv')
    df_val = pd.read_csv(path+'val_'+str(index)+'.csv')
    df_test = pd.read_csv(path+'test_'+str(index)+'.csv')
    
    # split train into hate and non-hate and take train_cnt
    # samples of each
    df_train_hate = df_train[df_train['Label'] == 1].sample(train_cnt,random_state=seed)
    df_train_non_hate = df_train[df_train['Label'] == 0].sample(train_cnt,random_state=seed)
    # concatenate hate and non_hate
    df_train = pd.concat([df_train_hate, df_train_non_hate])
    # shuffle the train data
    df_train = df_train.sample(frac=1).reset_index(drop=True)

    # clean data
    df_train=preprocess(df_train,args['isArabic'])
    df_val=preprocess(df_val,args['isArabic'])
    df_test=preprocess(df_test,args['isArabic'])

    return df_train, df_val, df_test

In [2]:
def train_part_hatebert(args,run,train_cnt,index,all_test_metrics,model_args,seed):
    model_name = args['model_name']
    model_args['name']=model_name+'_'+str(train_cnt)+'_'+str(index)+'_'+str(run)
    print("\tInitialising Model....")
    model = HateBERT_FewShot(model_args)
    print("\tLoading Dataset....")
    df_train, df_val, df_test = load_dataset_part(args,index,seed,train_cnt)
    print("\tTraining Starts....")
    train_metrics, test_metrics = model.run(model_args, 
                    df_train, df_val, df_test)

    # Save train metrics after generating path
    res_path=args['res_base_path']+model_name+'_'+model_args['name']
    save_metrics(res_path,train_metrics,"train")
    
    all_test_metrics.extend(test_metrics)

In [14]:
def run_part_hatebert(run_args,model_args,train_cnt):
    all_test_metrics=[]
    seeds = [42,43,44]
    for fold in [1, 2, 3, 4, 5]:
        print("Fold: ",fold)
        for run in [1,2,3]:
            print("Run: ",run)
            fix_random()
            train_part_hatebert(run_args,run,train_cnt,fold,all_test_metrics,model_args,seeds[run-1])
            print("Saving Test Metrics....")
            save_metrics(run_args['res_base_path']+run_args['model_name']+'_'+str(train_cnt),
                         all_test_metrics,"")

In [8]:
def preprocess(df,isArabic):
    
    X = df['Text']
    X_new=[]
    if(isArabic):
        prep = ArabertPreprocessor('bert-base-arabertv02')
        for text in tqdm(X):
            text = prep.preprocess(text)
            X_new.append(text)
    else:
        processer = Data_Preprocessing()
        for text in tqdm(X):
            text= processer.removeEmojis(text)
            text = processer.removeUrls(text)
            text=processer.removeSpecialChar(text)
            X_new.append(text)

    df['Text']=X_new
    return df 

In [9]:
def save_metrics(path,metrics,which):
    df = pd.DataFrame(metrics)
    df.to_csv(path+"_"+which+".csv")

In [10]:
def fix_random(seed_val=42):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

## Arabic 

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Let-Mi/',
    'train_cnt':256,
    'res_base_path': 'Results/Let-Mi/HateBert_fewData_fewShot/',
    'model_save_path': 'Saved_Models/Let-Mi/',
    'isArabic': True,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "Hate-speech-CNERG/dehatebert-mono-english",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': True,
        'model_path': "",
    }
run_part_hatebert(run_args,model_args)

Fold:  1
Run:  1
	Initialising Model....
	Loading Dataset....


  0%|          | 0/64 [00:00<?, ?it/s]

	Training Starts....


100%|██████████| 1047/1047 [00:00<00:00, 4979.97it/s]



======== Epoch 1 / 10 ========

Training...


...

  5%|▍         | 3/65 [00:00<00:02, 28.60it/s]

avg_train_loss 0.5327553823590279
train_f1Score 0.75
train_accuracy 0.78125

Running Validation...


  2%|▏         | 3/130 [00:00<00:04, 28.67it/s]

Validation Loss:  0.6133374149982745
Validation Accuracy:  0.6865384615384615
Best mF1Score....


 58%|█████▊    | 75/130 [00:02<00:01, 28.46it/s]

## Spanish

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-Spanish/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-Spanish/HateBert_fewData_fewShot/',
    'model_save_path': 'Saved_Models/AMI-Spanish/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "Hate-speech-CNERG/dehatebert-mono-english",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
    }
run_part_hatebert(run_args,model_args)

## Italian

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-2020/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-2020/HateBert_fewData_fewShot/',
    'model_save_path': 'Saved_Models/AMI-2020/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "Hate-speech-CNERG/dehatebert-mono-english",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
    }
for train_cnt in [64,128,256,512]:
    print("Train Count: ",train_cnt)
    run_part_hatebert(run_args,model_args,train_cnt)

Train Count:  64
Fold:  1
Run:  1
	Initialising Model....
	Loading Dataset....


  0%|          | 0/991 [00:00<?, ?it/s]

	Training Starts....


  0%|          | 0/16 [00:00<?, ?it/s]


======== Epoch 1 / 10 ========

Training...


...

  2%|▏         | 2/123 [00:00<00:08, 13.91it/s]

avg_train_loss 0.6201469898223877
train_f1Score 0.6333333333333334
train_accuracy 0.65625

Running Validation...


  1%|          | 2/247 [00:00<00:17, 13.64it/s]

Validation Loss:  0.6632768674110009
Validation Accuracy:  0.6178861788617886
Best mF1Score....


 37%|███▋      | 92/247 [00:06<00:11, 13.52it/s]

## Hindi

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Shared_Task_hin/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_hin/HateBert_fewData_fewShot/',
    'model_save_path': 'Saved_Models/Shared_Task_hin/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "Hate-speech-CNERG/dehatebert-mono-english",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
    }
for train_cnt in [64,128,256,512]:
    print("Train Count: ",train_cnt)
    run_part_hatebert(run_args,model_args,train_cnt)

## Bengali

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Shared_Task_iben/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_iben/HateBert_fewData_fewShot/',
    'model_save_path': 'Saved_Models/Shared_Task_iben/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "Hate-speech-CNERG/dehatebert-mono-english",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
    }
for train_cnt in [64,128,256,512]:
    print("Train Count: ",train_cnt)
    run_part_hatebert(run_args,model_args,train_cnt)

## Arabic Translated

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Let-Mi_Translated/',
    'train_cnt':256,
    'res_base_path': 'Results/Let-Mi/translated_HateBert_fewData_fewShot/',
    'model_save_path': 'Saved_Models/Let-Mi/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "Hate-speech-CNERG/dehatebert-mono-english",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
    }
run_part_hatebert(run_args,model_args)

## Italian Translated

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-2020_Translated/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-2020/translated_HateBert_fewData_fewShot/',
    'model_save_path': 'Saved_Models/AMI-2020/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "Hate-speech-CNERG/dehatebert-mono-english",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
    }
run_part_hatebert(run_args,model_args)

## Spanish Translated

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-Spanish_Translated/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-Spanish/translated_HateBert_fewData_fewShot/',
    'model_save_path': 'Saved_Models/AMI-Spanish/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "Hate-speech-CNERG/dehatebert-mono-english",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
    }
for train_cnt in [64,128,256,512]:
    print("Train Count: ",train_cnt)
    run_part_hatebert(run_args,model_args,train_cnt)

Train Count:  64
Fold:  1
Run:  1
	Initialising Model....



  0%|          | 0/128 [00:00<?, ?it/s]

	Loading Dataset....


  0%|          | 0/660 [00:00<?, ?it/s]

	Training Starts....


  0%|          | 0/16 [00:00<?, ?it/s]


======== Epoch 1 / 10 ========

Training...


 62%|██████▎   | 10/16 [00:03<00:01,  3.16it/s]

# One Shot Learning

Here we are checking the performance of the model trained on the English Dataset on other Datasets and their translated versions without any finetuning

## Utility Functions

In [2]:
def preprocess(df,isArabic):
    
    X = df['Text']
    X_new=[]
    if(isArabic):
        prep = ArabertPreprocessor('bert-base-arabertv02')
        for text in tqdm(X):
            text = prep.preprocess(text)
            X_new.append(text)
    else:
        processer = Data_Preprocessing()
        for text in tqdm(X):
            text= processer.removeEmojis(text)
            text = processer.removeUrls(text)
            text=processer.removeSpecialChar(text)
            X_new.append(text)

    df['Text']=X_new
    return df 

In [3]:
def one_shot_output(model_path,data_path,obj,args):
    saved_model=obj.load_model(model_path,args)
    device = torch.device(args['device'])
    saved_model=saved_model.to(device)
    
    df = pd.read_csv(data_path)
    
    # preprocessing
    df = preprocess(df,args['isArabic'])
    
    metrics = obj.run_test(saved_model,df,args)
    
    return metrics

### Arabic

In [48]:
DATA_PATH = "Data_Processed/Let-Mi/all.csv"
MODEL_PATH = "Saved_Models/Shared_Task_eng_1/best_bert_5_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 8.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': True,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [93]:
metrics

{'Test_accuracy': 0.5080521472392638,
 'Test_mF1Score': 0.3582618088308851,
 'Test_f1Score': 0.04821958456973294,
 'Test_auc': 0.5016587899692947,
 'Test_precision': 0.5284552845528455,
 'Test_recall': 0.025262339681305868,
 'Test_non_hatef1Score': 0.6683040330920372,
 'Test_non_recallScore': 0.9780552402572834,
 'Test_non_precisionScore': 0.5075593952483801,
 'Test_avg_loss': 1.5742717615665833}

### Italian

In [94]:
DATA_PATH = "Data_Processed/AMI-2020/all.csv"
MODEL_PATH = "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 8.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [95]:
metrics

{'Test_accuracy': 0.4881048387096774,
 'Test_mF1Score': 0.4206912490220223,
 'Test_f1Score': 0.22307221542227662,
 'Test_auc': 0.47740993693965234,
 'Test_precision': 0.41920644048303624,
 'Test_recall': 0.15196998123827393,
 'Test_non_hatef1Score': 0.618310282621768,
 'Test_non_recallScore': 0.8028498926410307,
 'Test_non_precisionScore': 0.5027502750275028,
 'Test_avg_loss': 1.142462940369883}

### Hindi 

In [96]:
DATA_PATH = "Data_Processed/Shared_Task_hin/all.csv"
MODEL_PATH = "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 8.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [97]:
metrics

{'Test_accuracy': 0.7639248704663213,
 'Test_mF1Score': 0.4962449171948354,
 'Test_f1Score': 0.12903225806451613,
 'Test_auc': 0.5193810380413171,
 'Test_precision': 0.3612040133779264,
 'Test_recall': 0.07854545454545454,
 'Test_non_hatef1Score': 0.8634575763251546,
 'Test_non_recallScore': 0.9602166215371798,
 'Test_non_precisionScore': 0.7844138165730815,
 'Test_avg_loss': 0.6544648472972484}

### Bengali 

In [98]:
DATA_PATH = "Data_Processed/Shared_Task_iben/all.csv"
MODEL_PATH = "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 8.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [99]:
metrics

{'Test_accuracy': 0.811491935483871,
 'Test_mF1Score': 0.4557765651587162,
 'Test_f1Score': 0.015789473684210523,
 'Test_auc': 0.5013903268056837,
 'Test_precision': 0.2571428571428571,
 'Test_recall': 0.008144796380090498,
 'Test_non_hatef1Score': 0.8957636566332218,
 'Test_non_recallScore': 0.9946358572312771,
 'Test_non_precisionScore': 0.814770998816968,
 'Test_avg_loss': 0.5455881597934872}

### Spanish

In [100]:
DATA_PATH = "Data_Processed/AMI-Spanish/all.csv"
MODEL_PATH = "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 8.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [101]:
metrics

{'Test_accuracy': 0.529126213592233,
 'Test_mF1Score': 0.4593614718614719,
 'Test_f1Score': 0.2651515151515152,
 'Test_auc': 0.5269658635159238,
 'Test_precision': 0.5907172995780591,
 'Test_recall': 0.17094017094017094,
 'Test_non_hatef1Score': 0.6535714285714287,
 'Test_non_recallScore': 0.8829915560916767,
 'Test_non_precisionScore': 0.518781006378455,
 'Test_avg_loss': 0.9959583759886547}

### Arabic Translated 

In [8]:
DATA_PATH = "Data_Processed/Let-Mi_Translated/all.csv"
MODEL_PATH = "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 8.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [9]:
metrics

{'Test_accuracy': 0.5362346625766872,
 'Test_mF1Score': 0.4070005913777842,
 'Test_f1Score': 0.1301690039554117,
 'Test_auc': 0.5300651178548914,
 'Test_precision': 0.8701923076923077,
 'Test_recall': 0.07034589972794403,
 'Test_non_hatef1Score': 0.6838321788001568,
 'Test_non_recallScore': 0.9897843359818388,
 'Test_non_precisionScore': 0.5223642172523961,
 'Test_avg_loss': 1.1201041296947223}

## Italian Translated

In [10]:
DATA_PATH = "Data_Processed/AMI-2020_Translated/all.csv"
MODEL_PATH = "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 8.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [11]:
metrics

{'Test_accuracy': 0.5763104838709677,
 'Test_mF1Score': 0.551683735968867,
 'Test_f1Score': 0.44660961158657014,
 'Test_auc': 0.5692229849069164,
 'Test_precision': 0.6061472480343102,
 'Test_recall': 0.353554304773817,
 'Test_non_hatef1Score': 0.6567578603511638,
 'Test_non_recallScore': 0.7848916650400156,
 'Test_non_precisionScore': 0.564588598708228,
 'Test_avg_loss': 1.121682591784385}

## Spanish Translated

In [12]:
DATA_PATH = "Data_Processed/AMI-Spanish_Translated/all.csv"
MODEL_PATH = "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 8.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [13]:
metrics

{'Test_accuracy': 0.6592839805825242,
 'Test_mF1Score': 0.6561029150729152,
 'Test_f1Score': 0.6891779684472737,
 'Test_auc': 0.6598918773225166,
 'Test_precision': 0.6303797468354431,
 'Test_recall': 0.76007326007326,
 'Test_non_hatef1Score': 0.6230278616985565,
 'Test_non_recallScore': 0.5597104945717732,
 'Test_non_precisionScore': 0.7024981074943225,
 'Test_avg_loss': 0.9711520723926211}

# Few Shot Learning

Here we are checking the performance of the model trained on the English Dataset on other Datasets and their translated versions with finetuning

## Arabic

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Let-Mi/',
    'train_cnt':256,
    'res_base_path': 'Results/Let-Mi/fewShot/',
    'model_save_path': 'Saved_Models/Let-Mi/',
    'isArabic': True,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': True,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run(run_args,model_args)

Fold:  1
	Initialising Model....


 38%|███▊      | 1400/3670 [00:00<00:00, 13992.71it/s]

	Loading Dataset....


 20%|██        | 747/3670 [00:00<00:00, 3705.02it/s]

	Training Starts....


...

  4%|▍         | 5/130 [00:00<00:02, 45.42it/s]

Validation Loss:  0.38866789375360195
Validation Accuracy:  0.8557692307692307
Testing Model....


  0%|          | 2/458 [00:00<00:44, 10.22it/s]


======== Epoch 3 / 10 ========

Training...


 77%|███████▋  | 352/458 [01:11<00:21,  4.89it/s]

## Spanish

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-Spanish/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-Spanish/fewShot/',
    'model_save_path': 'Saved_Models/AMI-Spanish/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run(run_args,model_args)

Fold:  1
	Initialising Model....


 10%|█         | 235/2317 [00:00<00:00, 2347.45it/s]

	Loading Dataset....


 27%|██▋       | 634/2317 [00:00<00:00, 3140.35it/s]

	Training Starts....


...

  0%|          | 1/289 [00:00<00:57,  5.02it/s]


======== Epoch 3 / 10 ========

Training...


  5%|▍         | 2/41 [00:00<00:01, 19.52it/s]

avg_train_loss 0.5192106330064754
train_f1Score 0.7799747580984433
train_accuracy 0.7737889273356401

Running Validation...


 78%|███████▊  | 32/41 [00:01<00:00, 19.92it/s]

## Hindi

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Shared_Task_hin/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_hin/fewShot/',
    'model_save_path': 'Saved_Models/Shared_Task_hin/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 4.5],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run(run_args,model_args)

Fold:  1
	Initialising Model....


  7%|▋         | 285/4327 [00:00<00:01, 2839.92it/s]

	Loading Dataset....


 17%|█▋        | 749/4327 [00:00<00:00, 3752.48it/s]

	Training Starts....


...

  3%|▎         | 5/154 [00:00<00:03, 45.14it/s]

Validation Loss:  0.47130339979738384
Validation Accuracy:  0.8814935064935064
Testing Model....


  0%|          | 1/540 [00:00<01:42,  5.28it/s]


======== Epoch 5 / 10 ========

Training...


 24%|██▍       | 130/540 [00:26<01:23,  4.93it/s]

## Arabic Translated

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Let-Mi_Translated/',
    'train_cnt':256,
    'res_base_path': 'Results/Let-Mi/translated_fewShot/',
    'model_save_path': 'Saved_Models/Let-Mi_Translated/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run(run_args,model_args)

Fold:  1
	Initialising Model....




  0%|          | 0/3670 [00:00<?, ?it/s]

  8%|▊         | 300/3670 [00:00<00:01, 2993.71it/s]

	Loading Dataset....




 16%|█▋        | 599/3670 [00:00<00:01, 2988.03it/s]

 24%|██▍       | 881/3670 [00:00<00:00, 2930.88it/s]

 31%|███▏      | 1148/3670 [00:00<00:00, 2847.27it/s]

 39%|███▉      | 1432/3670 [00:00<00:00, 2843.52it/s]

 47%|████▋     | 1723/3670 [00:00<00:00, 2861.46it/s]

 55%|█████▌    | 2026/3670 [00:00<00:00, 2909.73it/s]

 63%|██████▎   | 2305/3670 [00:00<00:00, 2872.64it/s]

 70%|███████   | 2584/3670 [00:00<00:00, 2846.57it/s]

 78%|███████▊  | 2865/3670 [00:01<00:00, 2834.45it/s]

 86%|████████▌ | 3145/3670 [00:01<00:00, 2821.82it/s]

100%|██████████| 3670/3670 [00:01<00:00, 2853.45it/s]


  0%|          | 0/523 [00:00<?, ?it/s]

100%|██████████| 523/523 [00:00<00:00, 2837.61it/s]


  0%|          | 0/1047 [00:00<?, ?it/s]

 28%|██▊       | 298/1047 [00:00<00:00, 2976.26it/s]

 56%|█████▌    | 584/1047 [00:00<00:00, 2938.63it/s]

100%|██████████| 1047/1047 [00:00<00:00, 2920.04it/s][A


  0%|          | 0/3670 [00:00<?, ?it/s]

 10%|█         | 370/3670 [00:00<00:00, 3699.99it

	Training Starts....


 20%|██        | 748/3670 [00:00<00:00, 3722.53it/s]

 30%|███       | 1115/3670 [00:00<00:00, 3705.42it/s]

 40%|████      | 1483/3670 [00:00<00:00, 3697.65it/s]

 50%|█████     | 1853/3670 [00:00<00:00, 3697.87it/s]

 61%|██████    | 2233/3670 [00:00<00:00, 3724.98it/s]

 71%|███████   | 2601/3670 [00:00<00:00, 3709.04it/s]

 81%|████████  | 2967/3670 [00:00<00:00, 3691.85it/s]

100%|██████████| 3670/3670 [00:00<00:00, 3712.13it/s]


  0%|          | 0/523 [00:00<?, ?it/s]

100%|██████████| 523/523 [00:00<00:00, 3614.72it/s]


  0%|          | 0/1047 [00:00<?, ?it/s]

 10%|█         | 108/1047 [00:00<00:01, 578.81it/s]

 44%|████▍     | 462/1047 [00:00<00:00, 772.62it/s]

100%|██████████| 1047/1047 [00:00<00:00, 2360.27it/s][A
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decod


======== Epoch 1 / 10 ========

Training...




  1%|          | 4/458 [00:00<00:43, 10.46it/s]

  1%|▏         | 6/458 [00:00<00:43, 10.51it/s]

  2%|▏         | 8/458 [00:00<00:42, 10.51it/s]

  2%|▏         | 10/458 [00:00<00:42, 10.51it/s]

  3%|▎         | 12/458 [00:01<00:42, 10.51it/s]

  3%|▎         | 14/458 [00:01<00:42, 10.51it/s]

  3%|▎         | 16/458 [00:01<00:42, 10.52it/s]

  4%|▍         | 18/458 [00:01<00:41, 10.52it/s]

  4%|▍         | 20/458 [00:01<00:41, 10.52it/s]

  5%|▍         | 22/458 [00:02<00:41, 10.52it/s]

  5%|▌         | 24/458 [00:02<00:41, 10.51it/s]

  6%|▌         | 26/458 [00:02<00:41, 10.52it/s]

  6%|▌         | 28/458 [00:02<00:40, 10.51it/s]

  7%|▋         | 30/458 [00:02<00:40, 10.50it/s]

  7%|▋         | 32/458 [00:03<00:40, 10.50it/s]

  7%|▋         | 34/458 [00:03<00:40, 10.50it/s]

  8%|▊         | 36/458 [00:03<00:40, 10.51it/s]

  8%|▊         | 38/458 [00:03<00:39, 10.51it/s]

  9%|▊         | 40/458 [00:03<00:39, 10.51it/s]

  9%|▉         | 42/458 [00:03<00:39, 10.51it/s]

 

## Spanish Translated

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-Spanish_Translated/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-Spanish/translated_fewShot/',
    'model_save_path': 'Saved_Models/AMI-Spanish/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run(run_args,model_args)

Fold:  1
	Initialising Model....




  0%|          | 0/2317 [00:00<?, ?it/s]

 10%|█         | 233/2317 [00:00<00:00, 2328.68it/s]

	Loading Dataset....




 20%|█▉        | 463/2317 [00:00<00:00, 2319.92it/s]

 30%|██▉       | 685/2317 [00:00<00:00, 2288.88it/s]

 40%|███▉      | 922/2317 [00:00<00:00, 2311.04it/s]

 50%|████▉     | 1156/2317 [00:00<00:00, 2316.28it/s]

 60%|█████▉    | 1390/2317 [00:00<00:00, 2321.27it/s]

 70%|███████   | 1628/2317 [00:00<00:00, 2335.13it/s]

 80%|████████  | 1862/2317 [00:00<00:00, 2336.10it/s]

100%|██████████| 2317/2317 [00:00<00:00, 2338.86it/s]


  0%|          | 0/330 [00:00<?, ?it/s]

100%|██████████| 330/330 [00:00<00:00, 2406.77it/s]


  0%|          | 0/660 [00:00<?, ?it/s]

 37%|███▋      | 243/660 [00:00<00:00, 2427.59it/s]

100%|██████████| 660/660 [00:00<00:00, 2406.82it/s]


  0%|          | 0/2317 [00:00<?, ?it/s]

 14%|█▎        | 314/2317 [00:00<00:00, 3133.75it/s]



	Training Starts....


 27%|██▋       | 630/2317 [00:00<00:00, 3140.52it/s]

 41%|████      | 951/2317 [00:00<00:00, 3159.36it/s]

 55%|█████▍    | 1268/2317 [00:00<00:00, 3160.67it/s]

 69%|██████▉   | 1599/2317 [00:00<00:00, 3202.50it/s]

 83%|████████▎ | 1921/2317 [00:00<00:00, 3202.79it/s]

100%|██████████| 2317/2317 [00:00<00:00, 3192.00it/s]


100%|██████████| 330/330 [00:00<00:00, 3352.61it/s]


  0%|          | 0/660 [00:00<?, ?it/s]

 51%|█████     | 334/660 [00:00<00:00, 3339.86it/s]

100%|██████████| 660/660 [00:00<00:00, 3282.68it/s]
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertF


======== Epoch 1 / 10 ========

Training...




  1%|▏         | 4/289 [00:00<00:27, 10.49it/s]

  2%|▏         | 6/289 [00:00<00:26, 10.50it/s]

  3%|▎         | 8/289 [00:00<00:26, 10.52it/s]

  3%|▎         | 10/289 [00:00<00:26, 10.54it/s]

  4%|▍         | 12/289 [00:01<00:26, 10.56it/s]

  5%|▍         | 14/289 [00:01<00:26, 10.56it/s]

  6%|▌         | 16/289 [00:01<00:25, 10.58it/s]

  6%|▌         | 18/289 [00:01<00:25, 10.58it/s]

  7%|▋         | 20/289 [00:01<00:25, 10.57it/s]

  8%|▊         | 22/289 [00:02<00:25, 10.57it/s]

  8%|▊         | 24/289 [00:02<00:25, 10.57it/s]

  9%|▉         | 26/289 [00:02<00:24, 10.57it/s]

 10%|▉         | 28/289 [00:02<00:24, 10.56it/s]

 10%|█         | 30/289 [00:02<00:24, 10.57it/s]

 11%|█         | 32/289 [00:03<00:24, 10.57it/s]

 12%|█▏        | 34/289 [00:03<00:24, 10.57it/s]

 12%|█▏        | 36/289 [00:03<00:23, 10.56it/s]

 13%|█▎        | 38/289 [00:03<00:23, 10.57it/s]

 14%|█▍        | 40/289 [00:03<00:23, 10.56it/s]

 15%|█▍        | 42/289 [00:03<00:23, 10.56it/s]

 

## Less data points few Shot

In [ ]:
def train_part(args,run,index,all_test_metrics,model_args,seed):
    model_name = args['model_name']
    model_args['name']=model_name+'_'+str(index)
    print("\tInitialising Model....")
    model = BERT_FewShot(model_args)
    print("\tLoading Dataset....")
    df_train, df_val, df_test = load_dataset_part(args,index,seed)
    print("\tTraining Starts....")
    train_metrics, test_metrics = model.run(model_args, 
                    df_train, df_val, df_test)

    # Save train metrics after generating path
    res_path=args['res_base_path']+model_name+'_'+model_args['name']
    save_metrics(res_path,train_metrics,"train")
    
    all_test_metrics.extend(test_metrics)

In [ ]:
def run_part(run_args,model_args):
    all_test_metrics=[]
    seeds = [42,43,44]
    for fold in [1, 2, 3, 4, 5]:
        print("Fold: ",fold)
        for run in [1,2,3]:
            print("Run: ",run)
            fix_random()
            train_part(run_args,run,fold,all_test_metrics,model_args,seeds[run-1])
            print("Saving Test Metrics....")
            save_metrics(run_args['res_base_path']+run_args['model_name'],
                         all_test_metrics,"")

## Arabic few data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Let-Mi/',
    'train_cnt':256,
    'res_base_path': 'Results/Let-Mi/fewData_fewShot/',
    'model_save_path': 'Saved_Models/Let-Mi/',
    'isArabic': True,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': True,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run_part(run_args,model_args)

Fold:  1
Run:  1
	Initialising Model....




100%|██████████| 64/64 [00:00<00:00, 13248.87it/s]


100%|██████████| 523/523 [00:00<00:00, 13902.51it/s]


100%|██████████| 1047/1047 [00:00<00:00, 14241.86it/s]


100%|██████████| 64/64 [00:00<00:00, 3756.86it/s]


  0%|          | 0/523 [00:00<?, ?it/s]



	Loading Dataset....
	Training Starts....


100%|██████████| 523/523 [00:00<00:00, 3679.62it/s]


  0%|          | 0/1047 [00:00<?, ?it/s]

 37%|███▋      | 383/1047 [00:00<00:00, 3826.12it/s]

100%|██████████| 1047/1047 [00:00<00:00, 3811.86it/s][A
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect


======== Epoch 1 / 10 ========

Training...


...



  8%|▊         | 10/130 [00:00<00:02, 44.85it/s]

 12%|█▏        | 15/130 [00:00<00:02, 44.75it/s]

 15%|█▌        | 20/130 [00:00<00:02, 44.60it/s]

 19%|█▉        | 25/130 [00:00<00:02, 44.51it/s]

 23%|██▎       | 30/130 [00:00<00:02, 44.60it/s]

 27%|██▋       | 35/130 [00:00<00:02, 44.53it/s]

 31%|███       | 40/130 [00:00<00:02, 44.67it/s]

 35%|███▍      | 45/130 [00:01<00:01, 44.48it/s]

 38%|███▊      | 50/130 [00:01<00:01, 44.53it/s]

 42%|████▏     | 55/130 [00:01<00:01, 44.58it/s]

 46%|████▌     | 60/130 [00:01<00:01, 44.61it/s]

 50%|█████     | 65/130 [00:01<00:01, 44.63it/s]

 54%|█████▍    | 70/130 [00:01<00:01, 44.63it/s]

 58%|█████▊    | 75/130 [00:01<00:01, 44.67it/s]

 62%|██████▏   | 80/130 [00:01<00:01, 44.69it/s]

 65%|██████▌   | 85/130 [00:01<00:01, 44.66it/s]

 69%|██████▉   | 90/130 [00:02<00:00, 44.66it/s]

 73%|███████▎  | 95/130 [00:02<00:00, 44.65it/s]

 77%|███████▋  | 100/130 [00:02<00:00, 44.38it/s]

 81%|████████  | 105/130 [00:02<00:00, 44.35it/


======== Epoch 6 / 10 ========

Training...




 50%|█████     | 4/8 [00:00<00:00, 10.34it/s]

 75%|███████▌  | 6/8 [00:00<00:00, 10.29it/s]

100%|██████████| 8/8 [00:00<00:00, 10.22it/s]


  0%|          | 0/65 [00:00<?, ?it/s]

  8%|▊         | 5/65 [00:00<00:01, 46.18it/s]

avg_train_loss 0.40898991748690605
train_f1Score 0.8
train_accuracy 0.828125

Running Validation...




 15%|█▌        | 10/65 [00:00<00:01, 45.75it/s]

 23%|██▎       | 15/65 [00:00<00:01, 45.20it/s]

 31%|███       | 20/65 [00:00<00:01, 44.72it/s]

 38%|███▊      | 25/65 [00:00<00:00, 44.59it/s]

 46%|████▌     | 30/65 [00:00<00:00, 44.73it/s]

 54%|█████▍    | 35/65 [00:00<00:00, 44.46it/s]

 62%|██████▏   | 40/65 [00:00<00:00, 44.32it/s]

 69%|██████▉   | 45/65 [00:01<00:00, 44.32it/s]

 77%|███████▋  | 50/65 [00:01<00:00, 44.41it/s]

 85%|████████▍ | 55/65 [00:01<00:00, 44.34it/s]

 92%|█████████▏| 60/65 [00:01<00:00, 44.43it/s]

100%|██████████| 65/65 [00:01<00:00, 44.43it/s]


  0%|          | 0/130 [00:00<?, ?it/s]



## Italian few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-2020/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-2020/fewData_fewShot/',
    'model_save_path': 'Saved_Models/AMI-2020/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run_part(run_args,model_args)

Fold:  1
Run:  1
	Initialising Model....




100%|██████████| 64/64 [00:00<00:00, 2419.84it/s]


  0%|          | 0/991 [00:00<?, ?it/s]

	Loading Dataset....




 34%|███▎      | 334/991 [00:00<00:00, 3332.80it/s]

100%|██████████| 991/991 [00:00<00:00, 3406.50it/s]


  0%|          | 0/1983 [00:00<?, ?it/s]

 18%|█▊        | 348/1983 [00:00<00:00, 3479.52it/s]

 36%|███▌      | 715/1983 [00:00<00:00, 3533.14it/s]

 54%|█████▍    | 1080/1983 [00:00<00:00, 3563.82it/s]

 73%|███████▎  | 1450/1983 [00:00<00:00, 3603.45it/s]

100%|██████████| 1983/1983 [00:00<00:00, 3575.19it/s]


100%|██████████| 64/64 [00:00<00:00, 4486.41it/s]


  0%|          | 0/991 [00:00<?, ?it/s]

 39%|███▉      | 385/991 [00:00<00:00, 3848.00it/s]



	Training Starts....


...



100%|██████████| 64/64 [00:00<00:00, 3644.99it/s]


  0%|          | 0/991 [00:00<?, ?it/s]

 34%|███▍      | 337/991 [00:00<00:00, 3367.08it/s]

	Loading Dataset....




100%|██████████| 991/991 [00:00<00:00, 3413.28it/s]


  0%|          | 0/1983 [00:00<?, ?it/s]

 17%|█▋        | 347/1983 [00:00<00:00, 3460.42it/s]

 36%|███▌      | 711/1983 [00:00<00:00, 3506.29it/s]

 54%|█████▍    | 1080/1983 [00:00<00:00, 3556.59it/s]

 73%|███████▎  | 1450/1983 [00:00<00:00, 3591.04it/s]

100%|██████████| 1983/1983 [00:00<00:00, 3562.60it/s]


100%|██████████| 64/64 [00:00<00:00, 3661.65it/s]


  0%|          | 0/991 [00:00<?, ?it/s]

 37%|███▋      | 368/991 [00:00<00:00, 3678.24it/s]



	Training Starts....


100%|██████████| 991/991 [00:00<00:00, 3810.67it/s]


  0%|          | 0/1983 [00:00<?, ?it/s]

 21%|██        | 407/1983 [00:00<00:00, 4065.02it/s]

 41%|████      | 817/1983 [00:00<00:00, 4075.38it/s]

 62%|██████▏   | 1225/1983 [00:00<00:00, 4075.64it/s]

100%|██████████| 1983/1983 [00:00<00:00, 4021.30it/s]
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT ex

## Spanish Few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-Spanish/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-Spanish/fewData_fewShot/',
    'model_save_path': 'Saved_Models/AMI-Spanish/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run_part(run_args,model_args)

## Hindi Few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Shared_Task_hin/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_hin/fewData_fewShot/',
    'model_save_path': 'Saved_Models/Shared_Task_hin/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run_part(run_args,model_args)

## Bengali Few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Shared_Task_iben/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_iben/fewData_fewShot/',
    'model_save_path': 'Saved_Models/Shared_Task_iben/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run_part(run_args,model_args)

## Arabic Translated Few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Let-Mi_Translated/',
    'train_cnt':256,
    'res_base_path': 'Results/Let-Mi/translated_fewData_fewShot/',
    'model_save_path': 'Saved_Models/Let-Mi/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run_part(run_args,model_args)

## Italian Translated Few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-2020_Translated/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-2020/translated_fewData_fewShot/',
    'model_save_path': 'Saved_Models/AMI-2020/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run_part(run_args,model_args)

## Spanish Translated Few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-Spanish_Translated/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-Spanish/translated_fewData_fewShot/',
    'model_save_path': 'Saved_Models/AMI-Spanish/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run_part(run_args,model_args)